In [1]:
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from transformers import AutoTokenizer, AdamWeightDecay, TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu
from datasets import load_dataset
# import evaluate
from transformers import GPT2Tokenizer, GPT2Model
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoModelWithLMHead, AutoTokenizer
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch 
from tensorflow.keras.preprocessing.text import Tokenizer
import string
device = torch.cuda

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/final_data.csv')

In [5]:
df = df.dropna(axis = 0)

In [6]:
df

,Unnamed: 0,Original Caption,Creative Caption 1,Creative Caption 2,Creative Caption 3
0,0,Beauty and the beast is this prince that's all...,"""Falling for the fairytale 🌹🥀","""A tale as old as time 🕰️💕","""Happily ever after with my beast 👸🦁"
1,1,A rare photograph of an actor signing autographs.,"""Capturing a moment in time 📸🎬","""Inked memories for a lifetime 🖋️🎞️","""A glimpse into history 🎥🔮"
2,2,A model walks the runway for a fashion show du...,"""Walking on air in couture flair ✨👠","""Stealing the show in style 👑🌟","""Strutting my stuff in the limelight 💃🏻✨"
3,3,Rock artist is the first artist to win a posth...,"""Remembering a musical genius 🎵🤘","""Legends never die 🎸🌟","""Living on through the power of music 🎶🎤"
4,4,Fat mouse with a piece of cheese.,"""Living my best life with some cheese 🧀😋","""Food coma goals 🐭💤","""Say cheese 📸🐁"
...,...,...,...,...,...
2939,1238,"""Investing your cash, rather than splashing it...","""Smart money moves - investing cash instead of...","""Wealth-building strategy - investing wisely, ...","""Financial success - investing cash for big ga..."
2940,1239,"""The camera monitors the seagull flying in a c...","""Bird's eye view - camera capturing the seagul...","""Sky-high beauty - camera capturing the majest...","""In-flight action - camera monitoring the seag..."
2941,1240,"""Underwater view of synchronized swimmers form...","""Synchronized elegance - underwater view of sw...","""Swimming harmony - stunning underwater view o...","""Aquatic artistry - breathtaking underwater vi..."
2942,1241,"""Flag with flagpole waving in the wind against...","""Patriotic pride - flag waving proudly in the ...","""Symbol of freedom - flag and flagpole swaying...","""Waving glory - flag flying high on a flagpole..."


In [7]:
train, test = train_test_split(df, test_size=0.1)

In [8]:
train, val = train_test_split(train, test_size = 0.1)

In [9]:
train1 = train.loc[:, ['Original Caption', 'Creative Caption 1']].reset_index(drop = True)
train2 = train.loc[:, ['Original Caption', 'Creative Caption 2']].reset_index(drop = True)
train3 = train.loc[:, ['Original Caption', 'Creative Caption 3']].reset_index(drop = True)

In [10]:
train1.rename(columns = {'Creative Caption 1': 'Creative Caption'}, inplace = True)
train2.rename(columns = {'Creative Caption 2': 'Creative Caption'}, inplace = True)
train3.rename(columns = {'Creative Caption 3': 'Creative Caption'}, inplace = True)

In [11]:
train = pd.concat([train1, train2, train3], ignore_index = True)

In [12]:
val1 = val.loc[:, ['Original Caption', 'Creative Caption 1']].reset_index(drop = True)
val2 = val.loc[:, ['Original Caption', 'Creative Caption 2']].reset_index(drop = True)
val3 = val.loc[:, ['Original Caption', 'Creative Caption 3']].reset_index(drop = True)

val1.rename(columns = {'Creative Caption 1': 'Creative Caption'}, inplace = True)
val2.rename(columns = {'Creative Caption 2': 'Creative Caption'}, inplace = True)
val3.rename(columns = {'Creative Caption 3': 'Creative Caption'}, inplace = True)

val = pd.concat([val1, val2, val3], ignore_index = True)

In [13]:
test1 = test.loc[:, ['Original Caption', 'Creative Caption 1']].reset_index(drop = True)
test2 = test.loc[:, ['Original Caption', 'Creative Caption 2']].reset_index(drop = True)
test3 = test.loc[:, ['Original Caption', 'Creative Caption 3']].reset_index(drop = True)

test1.rename(columns = {'Creative Caption 1': 'Creative Caption'}, inplace = True)
test2.rename(columns = {'Creative Caption 2': 'Creative Caption'}, inplace = True)
test3.rename(columns = {'Creative Caption 3': 'Creative Caption'}, inplace = True)

test = pd.concat([test1, test2, test3], ignore_index = True)

In [14]:
train = train.applymap(lambda x: '<SOS> ' + x + ' <EOS>' if type(x) == str else x)
val = val.applymap(lambda x: '<SOS> ' + x + ' <EOS>' if type(x) == str else x)
test = test.applymap(lambda x: '<SOS> ' + x + ' <EOS>' if type(x) == str else x)

In [15]:
train = Dataset.from_pandas(train.reset_index(drop = True))
val = Dataset.from_pandas(val.reset_index(drop = True))
test = Dataset.from_pandas(test.reset_index(drop = True))

In [16]:
max_len = 0
for pair in train:
  max_len = max(max_len, len(pair['Original Caption'].split()), len(pair['Creative Caption'].split()))

for pair in val:
  max_len = max(max_len, len(pair['Original Caption'].split()), len(pair['Creative Caption'].split()))

for pair in test:
  max_len = max(max_len, len(pair['Original Caption'].split()), len(pair['Creative Caption'].split()))

max_len

45

In [17]:
# self_tokenizer = Tokenizer(filters='', oov_token='<OOV>')
# self_tokenizer.fit_on_texts(train['Original Caption'])
# self_tokenizer.fit_on_texts(train['Creative Caption'])
# self_tokenizer.fit_on_texts(val['Original Caption'])
# self_tokenizer.fit_on_texts(val['Creative Caption'])
# self_tokenizer.fit_on_texts(test['Original Caption'])
# self_tokenizer.fit_on_texts(test['Creative Caption'])

In [18]:
checkpoint = "prithivida/parrot_paraphraser_on_T5"
tokenizer = AutoTokenizer.from_pretrained("prithivida/parrot_paraphraser_on_T5")

In [19]:
# new_tokens = set(self_tokenizer.word_index.keys()) - set(tokenizer.vocab.keys())

NameError: ignored

In [ ]:
# new_tokens_clean = set()
# for token in new_tokens:
#   new_tokens_clean.add(token.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
# temp = set()
# new_tokens = set()
# for tokens in new_tokens_clean:
#   if '<SOS>'  in tokens or '<EOS>' in tokens:
#     continue
#   if tokens.isalnum() == False:
#     temp.add(tokens)
#   else:
#     new_tokens.add(tokens)

In [ ]:
# new_tokens = list(new_tokens)

# for emoji in list(temp):
#   new_tokens.extend(emoji)

In [ ]:
# tokenizer.add_tokens(new_tokens)

In [20]:
special_tokens = ['<SOS>', '<EOS>', 'write creatively']
tokenizer.add_tokens(special_tokens, special_tokens=True)

3

In [21]:
prefix = "write creatively: "

def preprocess_function(examples):
  # print(examples)
  inputs = [prefix + example for example in examples['Original Caption']]
  targets = [example for example in examples['Creative Caption']]
  # print(inputs)
  # print(targets)
  model_inputs = tokenizer(inputs, max_length = max_len, truncation = True, padding = True, text_target = targets, return_tensors = 'pt')
  # print(type(model_inputs))
  return model_inputs

In [22]:
train = train.map(preprocess_function, batched = True)
val = val.map(preprocess_function, batched = True)

Map:   0%|          | 0/7101 [00:00<?, ? examples/s]

Map:   0%|          | 0/792 [00:00<?, ? examples/s]

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [24]:
# metric = evaluate.load('sacrebleu')
# rouge_metric = load_metric('rouge')

# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [[label.strip()] for label in labels]

#     return preds, labels


# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {"bleu": result["score"]}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v, 4) for k, v in result.items()}
#     return result

In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained("prithivida/parrot_paraphraser_on_T5").cuda()
model.resize_token_embeddings(len(tokenizer))

Embedding(32103, 768)

In [26]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Information Retrieval Project/saved_models/parrot_7k",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset = val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [27]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.362319
2,No log,1.075364
3,No log,1.029507
4,No log,1.006633
5,1.670300,1.003021
6,1.670300,1.001260
7,1.670300,1.001441
8,1.670300,1.001610
9,1.670300,1.001600
10,0.954500,1.001600


TrainOutput(global_step=1110, training_loss=1.2768929421364723, metrics={'train_runtime': 613.6463, 'train_samples_per_second': 115.718, 'train_steps_per_second': 1.809, 'total_flos': 3800575178496000.0, 'train_loss': 1.2768929421364723, 'epoch': 10.0})

In [72]:
temp = prefix + test['Original Caption'][11]
# temp = prefix + 'A man playing with his dog on the beach'
temp = tokenizer(temp, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt').input_ids.cuda()
temp = model.generate(temp)

In [73]:
temp

tensor([[    0, 32100,    96,   434,  2632,    53,    95,    28,     3,     9,
         10540, 18654,  7767,    55,     3,     2,     3, 32101,     1]],
       device='cuda:0')

In [74]:
tokenizer.batch_decode(temp)

['<pad><SOS> "Lighting up with a colorful shiny lamp! <unk> <EOS></s>']

In [75]:
test['Original Caption'][11]

'<SOS> vector illustration of a colorful shiny lamp for person with brown background . <EOS>'

In [76]:
test["Creative Caption"][11]

'<SOS> Light up your life with this beautiful lamp! 💡  <EOS>'

In [ ]:
9